# Amazon Product reviews playground
This note contains my playground for amazon products using Apache Livy to communicate with Spark

In [1]:
spark.version

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,None,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = 2.4.4


In [2]:
val base_path = "/media/ohtar10/Adder-Storage/datasets/amazon_products/reviews"
val metadata_path = base_path + "/metadata.json.gz"
val reviews_path = base_path + "/aggressive_dedup.json.gz"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

base_path: String = /media/ohtar10/Adder-Storage/datasets/amazon_products/reviews
metadata_path: String = /media/ohtar10/Adder-Storage/datasets/amazon_products/reviews/metadata.json.gz
reviews_path: String = /media/ohtar10/Adder-Storage/datasets/amazon_products/reviews/aggressive_dedup.json.gz


In [3]:
val metadata_df = spark.read.json(metadata_path)
metadata_df.createOrReplaceTempView("product_metadata")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

metadata_df: org.apache.spark.sql.DataFrame = [_corrupt_record: string, asin: string ... 8 more fields]


In [4]:
%%sql
select * from product_metadata limit 20

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/ohtar10/miniconda3/envs/wtsp/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()

We can observe the categories are encoded as array of arrays, and that a product might belong to more than one category. Let's first get rid of the first dimension of the category array and explore the different categories in case we want to get rid of redundant ones.

In [5]:
import spark.implicits._
import org.apache.spark.sql.functions._

//We need to double explode the records to get the categories separated.
metadata_df.select($"asin", 
                   explode($"categories").as("categories"), 
                   $"description").
    select($"asin", 
           explode($"categories").as("category"), 
           $"description").createOrReplaceTempView("product_metadata")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
import org.apache.spark.sql.functions._


In [6]:
%%sql
select distinct category from product_metadata

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

There are more than 2500 categoroes! this can become highly inpractical for experimentation. We need to reduce the amount of categories so we can operate them easier. We might want to discard those with less than $n$ products associated with them, and possibly merge some of them.

Let's start by working with the top 50 categories in terms of amount of products.

In [12]:
%%sql
select category, count(asin) as products
from product_metadata
group by category
having count(asin) > 10000
order by products desc
limit 50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

We can observe a good amount of products for these categories. We can also observe that categories such as 'Movies' and 'Movies & TV' can be merged togheter since they talk about similar things.

Another important aspect is that some categories seem to be related with a particular genere of something, this is the case of "classical" for example, it could be movies, music or books, a second category might be needed to differentiate it. For now, we'll stick with parent or root categories that has the capability of aggregate something.

In [ ]:
val catAlike = Map("Movies & TV" -> Array("Movies", "Movies & TV"),
                  "")